In [ ]:
import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# SAP EWM API base URL and API key (replace with your actual base URL and API key)
BASE_URL = 'https://your_sap_ewm_api_base_url'
API_KEY = 'your_sap_ewm_api_key'
ENDPOINT = '/inventory_accuracy_kpi'

# Google Sheet configuration
SERVICE_ACCOUNT_FILE = 'path/to/your_service_account_credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SHEET_ID = 'your_google_sheet_id'
RANGE = 'Sheet1'

# Function to request the inventory accuracy KPI from the SAP EWM API
def request_inventory_accuracy_kpi():
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }
    url = BASE_URL + ENDPOINT
    response = None

    for i in range(3):  # Retry the request up to 3 times
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = json.loads(response.text)
            return data['inventory_accuracy']
        except requests.exceptions.HTTPError as e:
            if response is not None and response.status_code == 429:
                wait_time = int(response.headers.get('Retry-After', 1))
                print(f"Rate limit exceeded, waiting for {wait_time} seconds.")
                time.sleep(wait_time)
            else:
                print(f"Error {e}: {response.status_code}")
        except Exception as e:
            print(f"Error: {e}")

    return None

# Function to find the first empty row in the specified column
def find_empty_row(sheet_range):
    credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, SCOPES)
    service = build('sheets', 'v4', credentials=credentials)

    try:
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID, range=sheet_range).execute()
        values = result.get('values', [])

        if not values:
            return 1
        else:
            return len(values) + 1
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# Function to write the inventory accuracy KPI to the Google Sheet
def write_inventory_accuracy_to_sheet(inventory_accuracy):
    credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, SCOPES)
    service = build('sheets', 'v4', credentials=credentials)

    empty_row = find_empty_row(RANGE)

    if empty_row is None:
        print("Error finding the empty row.")
        return

    cell = f"{RANGE}!A{empty_row}"

    body = {
        'values': [[inventory_accuracy]]
    }

    try:
        result = service.spreadsheets().values().update(
            spreadsheetId=SHEET_ID, range=cell,
            valueInputOption='RAW', body=body).execute()
        print(f"{result.get('updatedCells')} cells updated in the Google Sheet.")
    except HttpError as error:
        print(f"An error occurred: {error}")

# Main program
inventory_accuracy = request_inventory_accuracy_kpi()

if inventory_accuracy is not None:
    write_inventory_accuracy_to_sheet(inventory_accuracy)
